In [1]:
import os
import sys
import time
import argparse
import json
from tqdm import tqdm

import pandas as pd
import numpy as np
import pykeen
from pykeen.evaluation import rank_based_evaluator
import torch
from scipy import stats

from sheaf_kg.train_sheafE_betae import read_dataset, shuffle_datasets, dataset_to_device, sample_answers
import sheaf_kg.complex_functions as cf
# from sheaf_kg.complex_functions import test_batch

PyKEEN was unable to load dataset openbiolinkf1. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling
PyKEEN was unable to load dataset openbiolinkf2. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling


In [2]:
dataset = 'FB15k-237'
savename = 'TransE_1000epochs_64embdim_SoftplusLossloss_1234seed_20210519-1452'

savedir = f'/home/gebhart/projects/sheaf_kg/data/{dataset}'
complex_dataset_loc = f'/home/gebhart/projects/sheaf_kg/data/{dataset}-betae'

test_query_structures = ['1p','2p','3p','2i','3i','ip','pi']


In [3]:
model = torch.load(os.path.join(savedir, savename, 'trained_model.pkl'))

In [4]:
fun_map = {'1p':cf.L_p_traversal_transE,
           '2p':cf.L_p_traversal_transE,
           '3p':cf.L_p_traversal_transE,
           '2i':cf.L_i_traversal_transE,
           '3i':cf.L_i_traversal_transE,
           'ip':cf.L_ip_traversal_transE,
           'pi':cf.L_pi_traversal_transE
          }

def test_batch(model, test_data, model_inverses=False, test_batch_size=5,
                test_query_structures=test_query_structures,
                ks=[1,3,5,10]):
    with torch.no_grad():
        results = []
        for query_structure in test_query_structures:
            print('Running query : {}'.format(query_structure))
            all_avg_ranks = []
            cnt = 0
            num_test = len(test_data[query_structure]['answers'])
            for qix in tqdm(range(0, num_test//2, test_batch_size)):
                if num_test - qix == 1:
                    continue
                entities = test_data[query_structure]['entities'][qix:qix+test_batch_size]
                relations = test_data[query_structure]['relations'][qix:qix+test_batch_size]
                if model_inverses:
                    inverses = None
                else:
                    inverses = test_data[query_structure]['inverses'][qix:qix+test_batch_size]
                all_answers = test_data[query_structure]['answers'][qix:qix+test_batch_size]
                targets = torch.arange(model.entity_embeddings(indices=None).shape[0]).to(model.device)
                Q = fun_map[query_structure](model, entities, relations, targets, invs=inverses, p=1)
                answer_lens = np.array([len(a) for a in all_answers])
                max_len = answer_lens.max()
                for l in np.unique(answer_lens):
                    idxs = np.where(answer_lens == l)[0]
                    answers = [all_answers[j] for j in idxs]
                    filter_fix = torch.arange(l)
                    for aix in range(len(idxs)):
                        ranks = rank_based_evaluator.compute_rank_from_scores(Q[idxs[aix],answers[aix]].unsqueeze(1), Q[idxs[aix],:].unsqueeze(0))
                        avg_rank = (torch.sort(ranks['realistic'].cpu(), dim=0)[0] - filter_fix).numpy()
                        all_avg_ranks.append(avg_rank)
            all_avg_ranks = np.concatenate(all_avg_ranks)
            rd = {k: np.mean(all_avg_ranks <= k) for k in ks}
            mrr = np.reciprocal(stats.hmean(all_avg_ranks))
            rd['mrr'] = mrr if isinstance(mrr, float) else mrr[0]
            # rd['mr'] = np.mean(all_avg_ranks)
            results.append(rd)

        df = pd.DataFrame(results, index=test_query_structures)
        return df

In [5]:
datasets = read_dataset(complex_dataset_loc)
datasets = dataset_to_device(shuffle_datasets(datasets), model.device)

In [6]:
extension_df = test_batch(model, datasets['test-easy'], model_inverses=True, test_query_structures=test_query_structures)
extension_df*100

  3%|▎         | 46/1631 [00:00<00:03, 457.90it/s]

Running query : 1p


  4%|▎         | 18/489 [00:00<00:02, 175.23it/s]

Running query : 2p


  3%|▎         | 13/487 [00:00<00:03, 124.91it/s]

...

Running query : ip


  8%|▊         | 33/429 [00:00<00:01, 320.33it/s]

Running query : pi


100%|██████████| 429/429 [00:01<00:00, 325.37it/s]


,1,3,5,10,mrr
1p,1.426633,3.780749,5.617795,8.997517,3.940811
2p,0.190643,0.410875,0.579794,0.880178,0.505845
3p,0.052731,0.127994,0.193501,0.333343,0.229416
2i,0.140580,0.412254,0.590782,0.904716,0.510498
3i,0.125003,0.434960,0.715579,1.147988,0.660997
ip,0.031967,0.119421,0.203450,0.431104,0.281726
pi,0.042757,0.099931,0.142687,0.307747,0.322684
